In [ ]:
#| default_exp core

# Source

> Source code for fastlite

## Metadata

In [ ]:
#| export
from dataclasses import dataclass, field, make_dataclass, fields, Field, is_dataclass, MISSING
from typing import Any,Union,Optional
from inspect import get_annotations

from fastcore.utils import *
from fastcore.xml import highlight
from fastcore.xtras import hl_md, dataclass_src
from sqlite_minutils.db import *
from sqlite_minutils.utils import rows_from_file,TypeTracker,Format
import types

try: from graphviz import Source
except ImportError: pass

In [ ]:
from IPython.display import Markdown
from fastcore.test import test_fail

In [ ]:
from fastcore.net import urlsave
url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
path = Path('chinook.sqlite')
if not path.exists(): urlsave(url, path)

In [ ]:
db = Database("chinook.sqlite")

In [ ]:
#| exports
class _Getter:
    "Abstract class with dynamic attributes providing access to DB objects"
    def __init__(self, db): self.db = db
    # NB: Define `__dir__` in subclass to get list of objects
    def __repr__(self): return ", ".join(dir(self))
    def __contains__(self, s): return (s if isinstance(s,str) else s.name) in dir(self)
    def __getitem__(self, idxs):
        if isinstance(idxs,str): return self.db.table(idxs)
        return [self.db.table(o) for o in idxs]
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError
        return self.db[k]

class _TablesGetter(_Getter):
    def __dir__(self): return self.db.table_names()

@patch(as_prop=True)
def t(self:Database): return _TablesGetter(self)

By returning a `_TablesGetter` we get a repr and auto-complete that shows all tables in the DB.

In [ ]:
dt = db.t
dt

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track

In [ ]:
artist = dt.Artist
artist

<Table Artist (ArtistId, Name)>

This also can be used to get multiple tables at once.

In [ ]:
dt['Album','Artist']

[<Table Album (AlbumId, Title, ArtistId)>, <Table Artist (ArtistId, Name)>]

In [ ]:
assert 'Artist' in dt
assert artist in dt
assert 'foo' not in dt

In [ ]:
#| exports
class _Col:
    def __init__(self, t, c): self.t,self.c = t,c
    def __str__(self):  return f'"{self.t}"."{self.c}"'
    def __repr__(self):  return self.c
    def __iter__(self): return iter(self.c)

class _ColsGetter:
    def __init__(self, tbl): self.tbl = tbl
    def __dir__(self): return map(repr, self())
    def __call__(self): return [_Col(self.tbl.name,o.name) for o in self.tbl.columns]
    def __contains__(self, s): return (s if isinstance(s,str) else s.c) in self.tbl.columns_dict
    def __repr__(self): return ", ".join(dir(self))

    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError
        return _Col(self.tbl.name, k)

@patch(as_prop=True)
def c(self:Table): return _ColsGetter(self)

@patch(as_prop=True)
def c(self:View): return _ColsGetter(self)

Column auto-complete and repr are much the same as tables.

In [ ]:
ac = artist.c
ac

ArtistId, Name

Columns stringify in a format suitable for including in SQL statements.

In [ ]:
print(f"select {ac.Name} ...")

select "Artist"."Name" ...


In [ ]:
#| exports
@patch
def __str__(self:Table): return f'"{self.name}"'

@patch
def __str__(self:View): return f'"{self.name}"'

Tables and views do the same.

In [ ]:
print(f"select {ac.Name} from {artist}")

select "Artist"."Name" from "Artist"


In [ ]:
assert 'Name' in ac
assert ac.Name in ac
assert 'foo' not in ac

## Queries and views

In [ ]:
#| exports
@patch
def q(self:Database, sql: str, params = None)->list:
    return list(self.query(sql, params=params))

This is a minor shortcut for interactive use.

In [ ]:
acdc = db.q(f"select * from {artist} where {ac.Name} like 'AC/%'")
acdc

[{'ArtistId': 1, 'Name': 'AC/DC'}]

In [ ]:
#| exports
def _get_flds(tbl): 
    return [(k, v|None, field(default=tbl.default_values.get(k,None)))
            for k,v in tbl.columns_dict.items()]

def _dataclass(self:Table, store=True, suf='')->type:
    "Create a `dataclass` with the types and defaults of this table"
    res = make_dataclass(self.name.title()+suf, _get_flds(self))
    flexiclass(res)
    if store: self.cls = res
    return res

Table.dataclass = _dataclass

In [ ]:
artist_dc = artist.dataclass()
art1_obj = artist_dc(**acdc[0])
art1_obj

Artist(ArtistId=1, Name='AC/DC')

You can get the definition of the dataclass using fastcore's `dataclass_src`:

In [ ]:
src = dataclass_src(artist_dc)
hl_md(src, 'python')

```python
@dataclass
class Artist:
    ArtistId: int | None = None
    Name: str | None = None

```

In [ ]:
#| exports
def all_dcs(db, with_views=False, store=True, suf=''):
    "dataclasses for all objects in `db`"
    return [o.dataclass(store=store, suf=suf) for o in db.tables + (db.views if with_views else [])]

In [ ]:
#| exports
def create_mod(db, mod_fn, with_views=False, store=True, suf=''):
    "Create module for dataclasses for `db`"
    mod_fn = str(mod_fn)
    if not mod_fn.endswith('.py'): mod_fn+='.py'
    with open(mod_fn, 'w') as f:
        print('from dataclasses import dataclass', file=f)
        print('from typing import Any,Union,Optional\n', file=f)
        for o in all_dcs(db, with_views, store=store, suf=suf): print(dataclass_src(o), file=f)

In [ ]:
create_mod(db, 'db_dc')

In [ ]:
#| eval: false
from db_dc import Track
Track(**dt.Track.get(1))

Track(TrackId=1, Name='For Those About To Rock (We Salute You)', AlbumId=1, MediaTypeId=1, GenreId=1, Composer='Angus Young, Malcolm Young, Brian Johnson', Milliseconds=343719, Bytes=11170334, UnitPrice=0.99)

In [ ]:
#| exports
@patch
def __call__(
    self:(Table|View),
    where:str|None=None,  # SQL where fragment to use, for example `id > ?`
    where_args: Iterable|dict|NoneType=None, # Parameters to use with `where`; iterable for `id>?`, or dict for `id>:id`
    order_by: str|None=None, # Column or fragment of SQL to order by
    limit:int|None=None, # Number of rows to limit to
    offset:int|None=None, # SQL offset
    select:str = "*", # Comma-separated list of columns to select
    with_pk:bool=False, # Return tuple of (pk,row)?
    as_cls:bool=True, # Convert returned dict to stored dataclass?
    **kwargs)->list:
    "Shortcut for `rows_where` or `pks_and_rows_where`, depending on `with_pk`"
    f = getattr(self, 'pks_and_rows_where' if with_pk else 'rows_where')
    xtra = getattr(self, 'xtra_id', {})
    if xtra:
        xw = ' and '.join(f"[{k}] = {v!r}" for k,v in xtra.items())
        where = f'{xw} and {where}' if where else xw
    res = f(where=where, where_args=where_args, order_by=order_by, limit=limit, offset=offset, select=select, **kwargs)
    if as_cls and hasattr(self,'cls'):
        if with_pk: res = ((k,self.cls(**v)) for k,v in res)
        else: res = (self.cls(**o) for o in res)
    return list(res)

This calls either `rows_where` (if `with_pk`) or `with_pk` (otherwise). If `dataclass(store=True)` has been called, then if `as_cls` rows will be returned as dataclass objects.

In [ ]:
artist(limit=2)

[Artist(ArtistId=1, Name='AC/DC'), Artist(ArtistId=2, Name='Accept')]

If `with_pk` then tuples are returns with PKs 1st.

In [ ]:
artist(with_pk=True, limit=2)

[(1, Artist(ArtistId=1, Name='AC/DC')), (2, Artist(ArtistId=2, Name='Accept'))]

In [ ]:
artist.get(1)

{'ArtistId': 1, 'Name': 'AC/DC'}

In [ ]:
album = dt.Album

acca_sql = f"""select {album}.*
from {album} join {artist} using (ArtistId)
where {ac.Name} like 'AC/%'"""

In [ ]:
hl_md(acca_sql, 'sql')

```sql
select "Album".*
from "Album" join "Artist" using (ArtistId)
where "Artist"."Name" like 'AC/%'
```

In [ ]:
db.q(acca_sql)

[{'AlbumId': 1,
  'Title': 'For Those About To Rock We Salute You',
  'ArtistId': 1},
 {'AlbumId': 4, 'Title': 'Let There Be Rock', 'ArtistId': 1}]

In [ ]:
db.create_view("AccaDaccaAlbums", acca_sql, replace=True)

<Database <sqlite3.Connection object>>

In [ ]:
#| exports
class _ViewsGetter(_Getter):
    def __dir__(self): return self.db.view_names()

@patch(as_prop=True)
def v(self:Database): return _ViewsGetter(self)

In [ ]:
dv = db.v
dv

AccaDaccaAlbums

In [ ]:
dv.AccaDaccaAlbums()

[{'AlbumId': 1,
  'Title': 'For Those About To Rock We Salute You',
  'ArtistId': 1},
 {'AlbumId': 4, 'Title': 'Let There Be Rock', 'ArtistId': 1}]

In [ ]:
#| export
@patch
def create(
    self: Database,
    cls=None,  # Dataclass to create table from
    name=None,  # Name of table to create
    pk='id',  # Column(s) to use as a primary key
    foreign_keys=None,  # Foreign key definitions
    defaults=None,  # Database table defaults
    column_order=None,  # Which columns should come first
    not_null=None,  # Columns that should be created as ``NOT NULL``
    hash_id=None,  # Column to be used as a primary key using hash
    hash_id_columns=None,  # Columns used when calculating hash
    extracts=None,  # Columns to be extracted during inserts
    if_not_exists=False,  # Use `CREATE TABLE IF NOT EXISTS`
    replace=False,  # Drop and replace table if it already exists
    ignore=True,  # Silently do nothing if table already exists
    transform=False,  # If table exists transform it to fit schema
    strict=False,  # Apply STRICT mode to table
):
    "Create table from `cls`, default name to snake-case version of class name"
    flexiclass(cls)
    if name is None: name = camel2snake(cls.__name__)
    typs = {o.name: o.type for o in fields(cls)}
    res = self.create_table(
        name, typs, defaults=defaults,
        pk=pk, foreign_keys=foreign_keys, column_order=column_order, not_null=not_null,
        hash_id=hash_id, hash_id_columns=hash_id_columns, extracts=extracts, transform=transform,
        if_not_exists=if_not_exists, replace=replace, ignore=ignore, strict=strict)
    res.cls = cls
    return res

The class you pass to `create` is converted to a dataclass where any fields missing a default are defaulted to `None`.

In [ ]:
class Cat: id: int; name: str; age: int; city: str = "Unknown"
cats = db.create(Cat)
Cat(1)

Cat(id=1, name=UNSET, age=UNSET, city='Unknown')

To transform a table after creation, use the `.create()` method again, this time with the `transform` keyword set to `True`.


In [ ]:
class Cat: id: int; name: str; age: int; city: str = "Unknown"; breed: str = "Unknown"
cats = db.create(Cat, transform=True)
cats

<Table cat (id, name, age, city, breed)>

In [ ]:
Cat(1)

Cat(id=1, name=UNSET, age=UNSET, city='Unknown', breed='Unknown')

In [ ]:
print(cats.schema)

CREATE TABLE "cat" (
   [id] INTEGER PRIMARY KEY,
   [name] TEXT,
   [age] INTEGER,
   [city] TEXT,
   [breed] TEXT
)


In [ ]:
db.t.cat.drop()

In [ ]:
#| export
@patch
def import_file(self:Database, table_name, file, format=None, pk=None, alter=False):
    "Import path or handle `file` to new table `table_name`"
    if isinstance(file, str): file = file.encode()
    if isinstance(file, bytes): file = io.BytesIO(file)
    with maybe_open(file) as fp: rows, format_used = rows_from_file(fp, format=format)
    tracker = TypeTracker()
    rows = tracker.wrap(rows)
    tbl = self[table_name]
    tbl.insert_all(rows, alter=alter)
    tbl.transform(types=tracker.types)
    if pk: tbl.transform(pk=pk)
    return tbl

This uses [`sqlite_utils.utils.rows_from_file`](https://sqlite-utils.datasette.io/en/stable/reference.html#sqlite-utils-utils-rows-from-file) to load the file.

In [ ]:
db = Database(":memory:")
csv1 = "id,name,age\n1,Alice,30\n2,Bob,25"
csv2 = "id,name,age\n3,Charlie,35\n4,David,40"
csv3 = "id,name,age,city\n5,Eve,45,New York"

# import file to new table
tbl = db.import_file("people", csv1)
assert len(tbl()) == 2

# import file to existing table (same schema)
tbl = db.import_file("people", csv2)
assert len(tbl()) == 4

# import file to existing table (schema change fails)
test_fail(lambda: db.import_file("people", csv3),contains='city')

# import file to existing table (schema change succeeds)
assert 'city' not in tbl.c
tbl = db.import_file("people", csv3, alter=True)
assert 'city' in tbl.c

print(tbl())
tbl.drop()

[{'id': 1, 'name': 'Alice', 'age': 30, 'city': None}, {'id': 2, 'name': 'Bob', 'age': 25, 'city': None}, {'id': 3, 'name': 'Charlie', 'age': 35, 'city': None}, {'id': 4, 'name': 'David', 'age': 40, 'city': None}, {'id': 5, 'name': 'Eve', 'age': 45, 'city': 'New York'}]


## Database diagrams

(Requires graphviz.)

In [ ]:
fk = album.foreign_keys[0]
fk

ForeignKey(table='Album', column='ArtistId', other_table='Artist', other_column='ArtistId')

In [ ]:
#| exports
def _edge(tbl):
    return "\n".join(f"{fk.table}:{fk.column} -> {fk.other_table}:{fk.other_column};"
                     for fk in tbl.foreign_keys)

def _row(col):
    xtra = " 🔑" if col.is_pk else ""
    bg = ' bgcolor="#ffebcd"' if col.is_pk else ""
    return f'    <tr><td port="{col.name}"{bg}>{col.name}{xtra}</td></tr>'

def _tnode(tbl):
    rows = "\n".join(_row(o) for o in tbl.columns)
    res = f"""<table cellborder="1" cellspacing="0">
    <tr><td bgcolor="lightgray">{tbl.name}</td></tr>
{rows}
  </table>"""
    return f"{tbl.name} [label=<{res}>];\n"

In [ ]:
#| exports
def diagram(tbls, ratio=0.7, size="10", neato=False, render=True):
    layout = "\nlayout=neato;\noverlap=prism;\noverlap_scaling=0.5;""" if neato else ""
    edges  = "\n".join(map(_edge,  tbls))
    tnodes = "\n".join(map(_tnode, tbls))
    
    res = f"""digraph G {{
rankdir=LR;{layout}
size="{size}";
ratio={ratio};
node [shape=plaintext]

{tnodes}

{edges}
}}
"""
    return Source(res) if render else res

In [ ]:
diagram(db.tables)

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()